<a href="https://colab.research.google.com/github/zlqhem/mlapi/blob/main/torchscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
model_file = '/content/drive/MyDrive/w2/flutter/best.torchscript'
image_file = '/content/drive/MyDrive/w2/flutter/strawberry-healthy.png'
img=image_file

classes = np.array([
  'Tomato Healthy',
  'Strawberry Healthy',
  'Lettuce Healthy',
  'Strawberry Ashy Mold',
  'Strawberry White Powdery Mildew',
  'Lettuce Bacterial Head Rot',
  'Lettuce Bacterial Wilt',
  'Tomato Leaf Mold',
  'Tomato Yellow Leaf Curl Virus',
])

In [ ]:
import torch
print(torch.__version__)

2.1.0+cu121


In [ ]:
!ls -al /content/drive/MyDrive/w2/flutter/best.torchscript

-rw------- 1 root root 103561564 Nov  4 01:43 /content/drive/MyDrive/w2/flutter/best.torchscript


In [ ]:
import time
s = time.time()
model = torch.jit.load(model_file, map_location=torch.device('cpu')).eval()
#model = torch.jit.load(model_file).eval()
e = time.time()
print (e-s)

0.26126956939697266


In [ ]:
import torch
from PIL import Image
from torchvision import models, transforms
import json
#import boto3
import logging
#import base64
from io import BytesIO
import torch.nn.functional as F
import numpy as np

preprocess = transforms.Compose([
    transforms.Resize(640),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def lambda_handler(event, context):
  # example
  response = predict(input_fn(image_file), model)
  return {
      'statusCode': 200,
      'body': json.dumps(response)
  }

def input_fn(image_file):
  im = Image.open(image_file).resize((640,640))
  #https://dev.to/andreygermanov/how-to-create-yolov8-based-object-detection-web-service-using-python-julia-nodejs-javascript-go-and-rust-4o8e#prepare_the_input
  # "We do not need Alpha channel in the image for YOLOv8 predictions. Let's remove it"
  im = im.convert('RGB')
  img_tensor = preprocess(im)
  img_tensor = img_tensor.unsqueeze(0)
  return img_tensor

def input_fn(image_file):
  im = Image.open(image_file).resize((640,640))
  #https://dev.to/andreygermanov/how-to-create-yolov8-based-object-detection-web-service-using-python-julia-nodejs-javascript-go-and-rust-4o8e#prepare_the_input
  # "We do not need Alpha channel in the image for YOLOv8 predictions. Let's remove it"
  im = im.convert('RGB')
  input = np.array(im)
  input = input.transpose(2,0,1)
  input = input.reshape(1,3,640,640)
  input = input/255.0
  return torch.Tensor(input)


def predict(img_tensor, model):
  predict_values = model(img_tensor)
  print(predict_values[0].shape)
  preds = F.softmax(predict_values, dim=1)
  conf_score, indx = torch.max(preds, dim=1)
  conf_score = conf_score.cpu().numpy()
  indx = indx.cpu().numpy()
  predict_class = classes[indx]
  response = {}
  response['class'] = str(predict_class)
  response['confidence'] = str(conf_score)
  return response

# test
lambda_handler(None, None)


torch.Size([13, 8400])


{'statusCode': 200,
 'body': '{"class": "[[\'Lettuce Healthy\' \'Lettuce Healthy\' \'Lettuce Healthy\' ...\\n  \'Strawberry Healthy\' \'Strawberry Healthy\' \'Tomato Healthy\']]", "confidence": "[[0.9999974  1.         1.         ... 1.         1.         0.99998224]]"}'}